Chargement de la base de donnée session8.db


In [21]:
import pandas as pd # chargement des bases de donnees
import sqlite3  # chargement/ ecriture dans les bases de donnees

import requests # chargement des donnees depuis l'API
from requests import get#

import pprint #

import os.path #lire des fichiers
if not os.path.isfile("session8.db"):
    import urllib.request #telecharger un fichier depuis une url
    urllib.request.urlretrieve("http://homepages.ulb.ac.be/~choffreu/session8.db", "session8.db")#telechargement du fichier depuis lurl specifie sous le nom de session8.db

SESSION8 = sqlite3.connect('session8.db') #creation de la connexion avec la BDD (rouge)
cursor_SESSION8 = SESSION8.cursor() 

query = '''
  SELECT name, gsector, ticker
  FROM Companies 
  WHERE ticker IN ("GOOGL", "AMZN", "FB", "AAPL", "IDMO", "FFR", "TECTP","XTN", "FCAN", "MSFT")
'''
information_from_session8 = pd.read_sql_query(query,SESSION8) 

print(information_from_session8)

##ADD 5 companies







                           name                     gsector ticker
0                  ALPHABET INC  Telecommunication Services  GOOGL
1                  FACEBOOK INC  Telecommunication Services     FB
2        TECTONIC FINANCIAL INC                         N/A  TECTP
3                AMAZON.COM INC      Consumer Discretionary   AMZN
4   SPDR S&P TRANSPORTATION ETF                         N/A    XTN
5  FIRST TR FTSE EPRA/NAREIT DV                         N/A    FFR
6  INVESCO S&P INTL DEV MTM ETF                         N/A   IDMO
7   FIRST TR CANADA ALPHADEX FD                         N/A   FCAN
8                     APPLE INC      Information Technology   AAPL


Intialisation de la BDD du Projet:

-Creation des tableaux

-Copie des infos de la BDD session8 dans la BDD du projet



In [0]:

PROJET = sqlite3.connect('PROJET.db')
cursor_PROJET = PROJET.cursor()

#########Tableau Companies#########


####Ticker est la cle primaire pour permettre que ce soit une cle secondaire dans Candlesticks et alerts
cursor_PROJET.execute("""
CREATE TABLE IF NOT EXISTS Companies(
  ticker TEXT PRIMARY KEY UNIQUE,
  gsector TEXT,
  name TEXT
)
""")
PROJET.commit()
for i in range(0, len(information_from_session8) ):
  cursor_PROJET.execute(
                        """INSERT INTO
                              Companies(ticker,gsector,name) 
                              VALUES   (  ? ,   ?   ,  ?   )
                        """,(
                              information_from_session8["ticker"][i],
                              information_from_session8["gsector"][i],
                              information_from_session8["name"][i] 
                            )
                      )
  
  PROJET.commit()

#########Tableau Directions#########
query='''
        CREATE TABLE IF NOT EXISTS Directions (
          direction TEXT PRIMARY KEY UNIQUE
        )
'''

cursor_PROJET.execute(query)
PROJET.commit()

directions=["max","min"]
for i in directions:
  cursor_PROJET.execute(
                        """INSERT INTO
                              Directions(direction) 
                              VALUES   ( ? )
                        """,(i,))
  
  PROJET.commit()

#########Tableau Sources#########
query="""
        CREATE TABLE IF NOT EXISTS Sources (
          source TEXT PRIMARY KEY UNIQUE
        )
      """
cursor_PROJET.execute(query)
PROJET.commit()
sources=["open","close","high","low"]
for i in sources:
  cursor_PROJET.execute(
                        """INSERT INTO
                              Sources(source) 
                              VALUES   ( ? )
                        """,( 
                             i, 
                            )
                      )
  
  PROJET.commit()

#########Tableau Candlesticks#########
query="""
        CREATE TABLE IF NOT EXISTS Candlesticks (
          id INTEGER PRIMARY KEY,
          open_price FLOAT,
          high_price FLOAT,
          close_price FLOAT,
          low_price FLOAT,
          volume INT,
          timestamp DATETIME,
          ticker TEXT,
          FOREIGN KEY (ticker) REFERENCES Companies (ticker)
        )
      """
cursor_PROJET.execute(query)
PROJET.commit()
#########Tableau Alerts#########
query="""
        CREATE TABLE IF NOT EXISTS Alerts (
          name TEXT PRIMARY KEY UNIQUE,
          time_start DATETIME,
          time_end DATETIME,
          start_value FLOAT,
          end_value FLOAT,
          source TEXT,
          direction TEXT,
          ticker TEXT,
          FOREIGN KEY (source) REFERENCES Sources (source)
          FOREIGN KEY (direction) REFERENCES Directions (direction)
          FOREIGN KEY (ticker) REFERENCES Companies (ticker)
        )
      """
cursor_PROJET.execute(query)
PROJET.commit()





Chargement des tickers

In [46]:

query = '''
SELECT *
FROM Companies 
'''
resultat_PROJET_COMPANIES = pd.read_sql_query(query,PROJET)
print(resultat_PROJET_COMPANIES)

  ticker                     gsector                          name
0  GOOGL  Telecommunication Services                  ALPHABET INC
1     FB  Telecommunication Services                  FACEBOOK INC
2  TECTP                         N/A        TECTONIC FINANCIAL INC
3   AMZN      Consumer Discretionary                AMAZON.COM INC
4    XTN                         N/A   SPDR S&P TRANSPORTATION ETF
5    FFR                         N/A  FIRST TR FTSE EPRA/NAREIT DV
6   IDMO                         N/A  INVESCO S&P INTL DEV MTM ETF
7   FCAN                         N/A   FIRST TR CANADA ALPHADEX FD
8   AAPL      Information Technology                     APPLE INC


Chargement des données depuis l'API

In [47]:
token = "bpog5jfrh5ra872e4vkg"
liste_des_actions = resultat_PROJET_COMPANIES["ticker"]
        
from datetime import datetime
#start_date = datetime(2019, 1, 1)
#start_timestamp = int(datetime.timestamp(start_date)) # <-- ca donne le nombre de secondes écoulées entre le 1/1/1970 à 00:00 et 1/1/2020 
#end_timestamp = int(datetime.timestamp(datetime.now())) #Prise des dates jusqua maintenant
start_date = datetime(2020, 1, 13) # <-- LUNDI 13 janvier 2020 
start_timestamp = int(datetime.timestamp(start_date)) # <-- ca donne le nombre de secondes écoulées entre le 1/1/1970 à 00:00 et 1/1/2020 

end_date = datetime(2020, 1, 17)   # <-- Vendredi 17 janvier 2020 NON COMPRIS
end_timestamp = int(datetime.timestamp(end_date))


companies_candle = {}

for i in range(0, len(liste_des_actions)):
      API_URL = 'https://finnhub.io/api/v1/stock/candle?symbol=' + liste_des_actions[i] + '&resolution=D&from=' + str(start_timestamp) + '&to=' + str(end_timestamp) + '&token=' + token
      
      REPONSE_DE_L_API = requests.get(API_URL).json()
      companies_candle[liste_des_actions[i]] = REPONSE_DE_L_API
pprint.pprint(companies_candle)

{'AAPL': {'c': [316.96, 312.68, 311.34, 315.24],
          'h': [317.07, 317.57, 315.5, 315.7],
          'l': [311.15, 312.17, 309.55, 312.09],
          'o': [311.64, 316.7, 311.85, 313.59],
          's': 'ok',
          't': [1578925800, 1579012200, 1579098600, 1579185000],
          'v': [30383000, 40488600, 30480900, 27207300]},
 'AMZN': {'c': [1891.3, 1869.4399, 1862.02, 1877.9399],
          'h': [1898, 1887.11, 1878.86, 1885.59],
          'l': [1880.8, 1858.55, 1855.09, 1866.02],
          'o': [1891.3101, 1885.88, 1872.25, 1882.99],
          's': 'ok',
          't': [1578925800, 1579012200, 1579098600, 1579185000],
          'v': [2780800, 3440900, 2896600, 2659500]},
 'FB': {'c': [221.91, 219.06, 221.15, 221.77],
        'h': [221.97, 222.38, 221.68, 222.63],
        'l': [219.21, 218.63, 220.14, 220.39],
        'o': [219.6, 221.61, 220.61, 222.57],
        's': 'ok',
        't': [1578925800, 1579012200, 1579098600, 1579185000],
        'v': [14463400, 13288900, 1003650

Remplissage du tableau Candlesticks


In [48]:
for i in companies_candle.keys():
  if companies_candle[i]["s"] == "ok":
    for k in range(len(companies_candle[i]["o"])):
      cursor_PROJET.execute(
                          """INSERT INTO
                                Candlesticks(open_price,high_price,close_price,low_price,volume,timestamp,ticker) 
                                VALUES   (?,?,?,?,?,?,?)
                          """,(
                                companies_candle[i]["o"][k],
                                companies_candle[i]["h"][k],
                                companies_candle[i]["c"][k],
                                companies_candle[i]["l"][k],
                                companies_candle[i]["v"][k],
                                companies_candle[i]["t"][k],
                                i
                                
                              )
                        )
    
    PROJET.commit()

query = '''
SELECT *
FROM Candlesticks 
'''
resultat_PROJET_CANDLESTICKS = pd.read_sql_query(query,PROJET)
print(resultat_PROJET_CANDLESTICKS)

    id  open_price  high_price  ...    volume   timestamp  ticker
0    1   1435.2500    1441.480  ...   1536500  1578925800   GOOGL
1    2   1440.0000    1442.630  ...   1303800  1579012200   GOOGL
2    3   1433.0200    1440.780  ...   1077500  1579098600   GOOGL
3    4   1445.4500    1450.700  ...   1304000  1579185000   GOOGL
4    5    219.6000     221.970  ...  14463400  1578925800      FB
5    6    221.6100     222.380  ...  13288900  1579012200      FB
6    7    220.6100     221.680  ...  10036500  1579098600      FB
7    8    222.5700     222.630  ...  10015300  1579185000      FB
8    9     10.2520      10.480  ...      2400  1578925800   TECTP
9   10     10.4800      10.480  ...         0  1579012200   TECTP
10  11     10.4900      10.600  ...      2000  1579098600   TECTP
11  12     10.5290      10.531  ...       600  1579185000   TECTP
12  13   1891.3101    1898.000  ...   2780800  1578925800    AMZN
13  14   1885.8800    1887.110  ...   3440900  1579012200    AMZN
14  15   1

![Texte alternatif…](https://imgur.com/xgvpxyx.png)